##  Fase3. Prediccion para nuevos samples 

### Input's 
- Modelos de red Transformer
- pickle de transformador 

Instalar Pytorch desde Conda Promt
- conda install pytorch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 cpuonly -c pytorch

In [7]:
!pip install --quiet transformers==4.16.2

In [9]:
!pip install --quiet pandas

In [10]:
!pip install --quiet numpy

In [15]:
pip install  --quiet nltk

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install --quiet BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install --quiet lxml

Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install --quiet scikit-learn

In [17]:
!pip install --quiet tensorflow

### Librerias

In [1]:
import pandas as pd 
import numpy as np
import pickle
###Librerias de transformador
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer,
    PreTrainedTokenizer
)
import torch
import tensorflow as tf
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import string
from Transformador.transformadorPreprocesamiento import Preprocesamiento
import tqdm as notebook_tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Juan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Carga de Transformer con Tourch 

In [2]:
class TextSummarizationPredict:
    """ Clase TextSummarizationPredict """
    
    def __init__(self) -> None:
        """ Inicia la clase TextSummarizationT5 """
        pass

    def load_model(
      self, model_type: str = "t5", model_dir: str = "outputs", use_gpu: bool = False
    ):
        """
          carga un punto de control para inferencia/predicción
          Args:
              model_type (str, optional): "t5".
              model_dir (str, optional): ruta al directorio del modelo. Default "outputs".
              use_gpu (bool, optional): if True, el modelo usa gpu para inferencia/predicción. 
                                        Default True.
        """
        if model_type == "t5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_dir}")

        if use_gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                raise "exception ---> no gpu found. set use_gpu=False, to use CPU"
        else:
            self.device = torch.device("cpu")

        self.model = self.model.to(self.device)

    def predict(
            self,
            source_text: str,
            max_length: int = 32,
            num_return_sequences: int = 1,
            num_beams: int = 2,
            top_k: int = 50,
            top_p: float = 0.95,
            do_sample: bool = True,
            repetition_penalty: float = 2.5,
            length_penalty: float = 1.0,
            early_stopping: bool = True,
            skip_special_tokens: bool = True,
            clean_up_tokenization_spaces: bool = True,
        ):
        
        """
            Genera predicción para el modelo T5
            Args:
              source_text (str): texto para generar predicción
              max_length (int, optional): longitud máxima del token de predicción.
              num_return_sequences (int, optional): número de predicciones a devolver. 
                                                    Default 1.
              num_beams (int, optional): numero de beams. Default 2.
              top_k (int, optional): Default 50.
              top_p (float, optional): Default 0.95.
              do_sample (bool, optional): Default True.
              repetition_penalty (float, optional): Defaults 2.5.
              length_penalty (float, optional): Defaults 1.0.
              early_stopping (bool, optional): Defaults True.
              skip_special_tokens (bool, optional): Defaults True.
              clean_up_tokenization_spaces (bool, optional): Defaults True.
            Returns:
              list[str]: returns predicción
        """
        input_ids = self.tokenizer.encode(source_text, return_tensors="pt", add_special_tokens=True)
        input_ids = input_ids.to(self.device)
        generated_ids = self.model.generate(
          input_ids=input_ids,
          num_beams=num_beams,
          max_length=max_length,
          repetition_penalty=repetition_penalty,
          length_penalty=length_penalty,
          early_stopping=early_stopping,
          top_p=top_p,
          top_k=top_k,
          num_return_sequences=num_return_sequences,
        )
        preds = [
            self.tokenizer.decode(
              g,
              skip_special_tokens=skip_special_tokens,
              clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )
            for g in generated_ids
        ]
        return preds

In [3]:
model = TextSummarizationPredict()

In [4]:
# load trained T5 model
model.load_model("t5","modelo/TextSummarizationT5/TextSummarizationT5-epoch-7-train-loss-0.4759-val-loss-1.2537", use_gpu=False)

### Preparación 

#### Preprocesamiento

In [5]:
def cargarPipeline(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    return pipeline

In [9]:
from Transformador.transformadorPreprocesamiento import Preprocesamiento
pipe = cargarPipeline('Transformador/pipePreprocesador')
print(pipe)
text = 'Hacia tiempo que no comiamos algo tan bueno y de tanta calidad. Si es cierto que el sitio se hace un poco pequeño y dias de fin de semana seguramente se llene mucho. Pero la comida es excelente, cada plato que pruebas, hace que quieras probar otro que saca a otros comensales. Un sitio muy recomendado.'
textProcesado = pipe.transform(text)
textProcesado.head()

Pipeline(steps=[('texto_preprocesado', Preprocesamiento())])


,Text,cleaned_text
0,Hacia tiempo que no comiamos algo tan bueno y ...,hacia tiempo que no comiamos algo tan bueno y ...


In [10]:
textProcesado['cleaned_text'][0]

'hacia tiempo que no comiamos algo tan bueno y de tanta calidad  si es cierto que el sitio se hace un poco pequeño y dias de fin de semana seguramente se llene mucho  pero la comida es excelente  cada plato que pruebas  hace que quieras probar otro que saca a otros comensales  un sitio muy recomendado '

### Predicción

In [11]:
text_to_summarize=textProcesado['cleaned_text'][0]
model.predict(text_to_summarize)

['comida es excelente']

### Clase para Predicción

In [2]:
class TextSummarizationPredict:
    """ Clase TextSummarizationPredict """
    
    def __init__(self) -> None:
        """ Inicia la clase TextSummarizationT5 """
        pass

    def load_model(
      self, model_type: str = "t5", model_dir: str = "outputs", use_gpu: bool = False
    ):
        """
          carga un punto de control para inferencia/predicción
          Args:
              model_type (str, optional): "t5".
              model_dir (str, optional): ruta al directorio del modelo. Default "outputs".
              use_gpu (bool, optional): if True, el modelo usa gpu para inferencia/predicción. 
                                        Default True.
        """
        if model_type == "t5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_dir}")

        if use_gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                raise "exception ---> no gpu found. set use_gpu=False, to use CPU"
        else:
            self.device = torch.device("cpu")

        self.model = self.model.to(self.device)

    def predict(
            self,
            source_text: str,
            max_length: int = 32,
            num_return_sequences: int = 1,
            num_beams: int = 2,
            top_k: int = 50,
            top_p: float = 0.95,
            do_sample: bool = True,
            repetition_penalty: float = 2.5,
            length_penalty: float = 1.0,
            early_stopping: bool = True,
            skip_special_tokens: bool = True,
            clean_up_tokenization_spaces: bool = True,
        ):
        
        """
            Genera predicción para el modelo T5
            Args:
              source_text (str): texto para generar predicción
              max_length (int, optional): longitud máxima del token de predicción.
              num_return_sequences (int, optional): número de predicciones a devolver. 
                                                    Default 1.
              num_beams (int, optional): numero de beams. Default 2.
              top_k (int, optional): Default 50.
              top_p (float, optional): Default 0.95.
              do_sample (bool, optional): Default True.
              repetition_penalty (float, optional): Defaults 2.5.
              length_penalty (float, optional): Defaults 1.0.
              early_stopping (bool, optional): Defaults True.
              skip_special_tokens (bool, optional): Defaults True.
              clean_up_tokenization_spaces (bool, optional): Defaults True.
            Returns:
              list[str]: returns predicción
        """
        input_ids = self.tokenizer.encode(source_text, return_tensors="pt", add_special_tokens=True)
        input_ids = input_ids.to(self.device)
        generated_ids = self.model.generate(
          input_ids=input_ids,
          num_beams=num_beams,
          max_length=max_length,
          repetition_penalty=repetition_penalty,
          length_penalty=length_penalty,
          early_stopping=early_stopping,
          top_p=top_p,
          top_k=top_k,
          num_return_sequences=num_return_sequences,
        )
        preds = [
            self.tokenizer.decode(
              g,
              skip_special_tokens=skip_special_tokens,
              clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )
            for g in generated_ids
        ]
        return preds

In [3]:
import pandas as pd 
import numpy as np
import pickle
###Librerias de transformador
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer,
    PreTrainedTokenizer
)
import torch
import tensorflow as tf
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import string
from Transformador.transformadorPreprocesamiento import Preprocesamiento
import tqdm as notebook_tqdm
from Transformador.transformadorPreprocesamiento import Preprocesamiento

class SummarizationPredict:
    """ Clase TextSummarizationPredict """
    
    def __init__(self) -> None:
        """ Inicia la clase TextSummarizationT5 """
        pass
   
    def cargarPipeline(self, nombreArchivo):
        with open(nombreArchivo+'.pickle', 'rb') as handle:
            pipeline = pickle.load(handle)
        return pipeline
    
    def limpiezaTextoParaPredict(self, texto):
        pipe = self.cargarPipeline('Transformador/pipePreprocesador')
        text = texto
        textProcesado = pipe.transform(text)
        return textProcesado['cleaned_text'][0]
    
    def predictSummarization(self, texto):
        model = TextSummarizationPredict()
        model.load_model("t5","modelo/TextSummarizationT5/TextSummarizationT5-epoch-7-train-loss-0.4759-val-loss-1.2537", use_gpu=False)
        text_to_summarize=self.limpiezaTextoParaPredict(texto)
        return(model.predict(text_to_summarize))
        
            

In [4]:
texto = 'Hacia tiempo que no comiamos algo tan bueno y de tanta calidad. Si es cierto que el sitio se hace un poco pequeño y dias de fin de semana seguramente se llene mucho. Pero la comida es excelente, cada plato que pruebas, hace que quieras probar otro que saca a otros comensales. Un sitio muy recomendado.'
model = SummarizationPredict().predictSummarization(texto)
print(model)

C:\Users\Juan\anaconda3\envs\Tesis-Env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


['comida es excelente']


In [5]:
texto = 'El servicio y la comida son muy buenos en este restaurante. Tienen un bar completo y un menú caro con delicias españolas. Dicho esto, debo advertir al lector las porciones no son abundantes y es caro. Fuimos allí para comer el mes pasado y realmente disfrutamos de la paella.'
model = SummarizationPredict().predictSummarization(texto)
print(model)

C:\Users\Juan\anaconda3\envs\Tesis-Env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


['servicio y comida muy buenos']


### Probando Modelo con nuevos comentarios 

- Los comentarios fueron tomados de: https://www.tripadvisor.com/Restaurant_Review-g294309-d1928308-Reviews-Restaurante_El_Jardin-Cuenca_Azuay_Province.html

In [10]:
comentarios = ['Un lugar muy bonito, con mucha decoración, agradable ambiente. La atención es muy demorada y la comida no es lo que se espera.',
               'La comida deliciosa! El ambiente hermoso, la atención excelente, y todo por precios razonables! Definitivamente recomendado!', 
               'El ambiente de lujo, la atención esmerada y con mucha amabilidad, la comida con una combinación de sencillez y que resalta cada sabor en la boca. No soy de comer postres, pero me dieron uno de crema de menta que es un real éxtasis.' ,
              'El restaurante El Jardin del Hotel Victoria, es un lugar que conserva una tradición de más de 40 años en Cuenca, pues este fue el primer restaurante de servicio completo que abrió en la ciudad y que se mantiene hasta la actualidad. Una carta bien lograda, con platos esenciales y bien servidos siguiendo principios culinarios de siempre. El servicio es muy bueno, y basta haber ido más d una vez que la relación con sus dueños y empleados se vuelve absolutamente familiar. La vista del Tomebamba y del jardín del Hotel representan un valor agregado enorme para las personas que buscan un remanso pacífico en sus labores. Muy recomendables las sugerencias diarias del chef y las propuestas gastronómicas de temporada. Actualmente se encuentra solo con servicio a domicilio, esperado que vuelva pronto a abrir sus puertas según comentan sus propietarios.',
              'Llegamos con muy altas expectativas.A las 13h45 ingresamos, nos tomaron el pedido a las 14h00, las entradas las trajeron a las 14h40, luego de eso los platos fuertes llegaron a la 15h25, y cuando llegó la comida fue muy decepcionante, platos desabridos que ni siquiera entran dentro de un gusto normal... finalmente las bebidas nunca llegaron. Uno de los meseros a mi parecer dio un servicio quemimportista.Sentí que boté el dinero a la basura.No volveré a este lugar, lástima que la vista es bonita.']

values = []
for comentario in comentarios:
    summary = SummarizationPredict().predictSummarization(comentario)
    values.append([comentario, summary[0]])

column_name = ['comentario','resumen generado']               
dfResultados = pd.DataFrame(values,columns=column_name)
dfResultados

C:\Users\Juan\anaconda3\envs\Tesis-Env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Juan\anaconda3\envs\Tesis-Env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Juan\anaconda3\envs\Tesis-Env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid resu

,comentario,resumen generado
0,"Un lugar muy bonito, con mucha decoración, agr...",atención agradable ambiente
1,"La comida deliciosa! El ambiente hermoso, la a...",ambiente hermoso
2,"El ambiente de lujo, la atención esmerada y co...",ambiente de lujo
3,"El restaurante El Jardin del Hotel Victoria, e...",restaurante servicio bueno
4,Llegamos con muy altas expectativas.A las 13h4...,comida fue muy decepcionante


In [13]:
pip install openpyxl

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
dfResultados.to_excel(r'TablaResultadosResumenesPredict.xlsx', index = False, header=True)